In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from pathlib import Path
import os
import glob
import numpy
from tqdm import tqdm
from datetime import datetime

In [2]:
#folder = 'D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-02-24/categories/*'

# Rename files in directory so that they have the correct file extension.
def converthtmlsr2(folder):
    """Changes file type to from xxxx to xxxx.html so that the files can be read in correctly"""
    for filename in glob.iglob(os.path.join(folder, 'ite*')):
      os.rename(filename, filename + '.html')

#converthtmlsr2(folder)

In [3]:
from_symbol = 'BTC'
to_symbol = 'USD'
exchange = 'Coindesk'
datetime_interval = 'day'

def download_data(exchange, datetime_interval):
    supported_intervals = {'minute', 'hour', 'day'}
    assert datetime_interval in supported_intervals,\
        'datetime_interval should be one of %s' % supported_intervals
    url = 'https://api.coindesk.com/v1/bpi/historical/close.json?start=2013-12-20&end=2014-11-05'
    request = requests.get(url)
    data = request.json()
    return data

raw_exchange = download_data(exchange, datetime_interval)
exchange = raw_exchange['bpi']
exchange

{'2013-12-20': 622.8418,
 '2013-12-21': 608.9638,
 '2013-12-22': 617.8158,
 '2013-12-23': 673.1677,
 '2013-12-24': 667.6203,
 '2013-12-25': 687.3557,
 '2013-12-26': 765.2067,
 '2013-12-27': 746.7232,
 '2013-12-28': 730.4715,
 '2013-12-29': 745.0712,
 '2013-12-30': 754.7645,
 '2013-12-31': 757.4953,
 '2014-01-01': 770.4357,
 '2014-01-02': 808.0485,
 '2014-01-03': 830.024,
 '2014-01-04': 858.9833,
 '2014-01-05': 940.0972,
 '2014-01-06': 951.3865,
 '2014-01-07': 810.5833,
 '2014-01-08': 859.9485,
 '2014-01-09': 860.895,
 '2014-01-10': 884.6667,
 '2014-01-11': 930.905,
 '2014-01-12': 873.2635,
 '2014-01-13': 857.9564,
 '2014-01-14': 851.828,
 '2014-01-15': 874.713,
 '2014-01-16': 847.3735,
 '2014-01-17': 828.222,
 '2014-01-18': 843.7583,
 '2014-01-19': 878.6807,
 '2014-01-20': 871.0483,
 '2014-01-21': 874.2885,
 '2014-01-22': 863.95,
 '2014-01-23': 854.3475,
 '2014-01-24': 825.1221,
 '2014-01-25': 861.8516,
 '2014-01-26': 880.1523,
 '2014-01-27': 814.5302,
 '2014-01-28': 833.9395,
 '2014-0

In [71]:
def cleanhtmlsr2(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Vendor Rating, Product Review Count, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = [i.text for i in title_raw] #re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  details_raw = raw_html.find_all('div', {'class' :'item_details'})
  details = [i.text for i in details_raw] #re.sub(cleantags, '', str(details_raw)).strip()
  details = [i.split('\n') for i in details]
  for i in range(0, len(details)):
    sellerid_full = []
#    origin_full = details
    sellerid_full = sellerid_full.append(details[i][1])
#    origin_full[i] = details[i][2]
#  sellerid = [i.split(': ', 1)[1] for i in sellerid_full]
#  origin_full = details[2::4]
#  origin = [i.split(': ', 1)[1] for i in origin_full]
#  destination_full = details[3::4]
#  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' :'price_big'})
  priceUSD = []
  priceBTC = str(price_raw).replace('[','').replace(']','').replace('<div class="price_big">฿','').replace('</div>','').split(', ')
  for i in range(0, len(priceBTC)): 
    try:
      float(priceBTC[i])
    except ValueError:
      priceBTC[i] = float(0.0)
  priceBTC = [float(i) for i in priceBTC] #New
  rating = []  # Seller Rating. Out of 5.
  reviews = [] # The number of customer reviews
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
#  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
#  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
#  df['BTCUSD'] = df['Date'].map(exchange)
#  df['PriceUSD'] = df['BTCUSD'] * df['PriceBTC'] # Price in American Dollars
#  df = df.drop('BTCUSD', axis=1)
  return sellerid_full

#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2013-12-20/categories/art/items.html')
#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-01-16/categories/drugs-dissociatives-dioscorea/items.html')
cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-07-17/categories/apparel/items.html')

In [72]:
def cleanhtmlsr2(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Vendor Rating, Product Review Count, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = [i.text for i in title_raw] #re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  details_raw = raw_html.find_all('div', {'class' :'item_details'})
  details = re.sub(cleantags, '', str(details_raw)).strip().split('\n  ')
  sellerid_full = details[1::4]
  sellerid = [i.split(': ', 1)[1] for i in sellerid_full]
  origin_full = details[2::4]
  origin = [i.split(': ', 1)[1] for i in origin_full]
  destination_full = details[3::4]
  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' :'price_big'})
  priceUSD = []
  priceBTC = str(price_raw).replace('[','').replace(']','').replace('<div class="price_big">฿','').replace('</div>','').split(', ')
  for i in range(0, len(priceBTC)): 
    try:
      float(priceBTC[i])
    except ValueError:
      priceBTC[i] = float(0.0)
  priceBTC = [float(i) for i in priceBTC] #New
  rating = []  # Seller Rating. Out of 5.
  reviews = [] # The number of customer reviews
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  df['BTCUSD'] = df['Date'].map(exchange)
  df['PriceUSD'] = df['BTCUSD'] * df['PriceBTC'] # Price in American Dollars
  df = df.drop('BTCUSD', axis=1)
  return df

#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2013-12-20/categories/art/items.html')
#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-01-16/categories/drugs-dissociatives-dioscorea/items.html')
cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-07-17/categories/apparel/items.html')

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,Ray Ban Tech RB3460 001 Aviator/Flip Out/ Sung...,FoxyGirl,61.1615,0.098232,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17
1,Ray Ban RB3025 Aviator Classic Sunglasses Replica,FoxyGirl,37.9626,0.060972,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17
2,Rolex - Watch Box (AAA Grade Replica),RepAAA,100.43,0.161301,None,None,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-07-17
3,Ray Ban RB3016 - W0365 | Clubmaster Sunglasses...,FoxyGirl,45.3438,0.072827,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17
4,Rolex - Submariner 2Tone YG/SS Black [Replica],RepAAA,149.635,0.24033,None,None,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-07-17
5,Chanel Ultra Ceramic Bracelet Replica,FoxyGirl,44.2897,0.071134,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17
6,Louis Vuitton Initials Damier Graphite Belt - ...,FoxyGirl,54.8344,0.08807,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17
7,Ray Ban RB3025 112 Aviator Flash Lenses Sungla...,FoxyGirl,37.9626,0.060972,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17
8,Nike Yeezy II Shoes Replica,FoxyGirl,100.178,0.160897,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17
9,Oakley Ducati Holbrook sunglasses replica,FoxyGirl,34.799,0.055891,None,None,China,Worldwide,Apparel,None,SilkRoad2,2014-07-17


In [73]:
def cleanhtmlsr2part2(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Vendor Rating, Product Review Count, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
#  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = [i.text for i in title_raw] #re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  sellerid_raw = raw_html.find_all('div', {'class' :'vendor'})
  sellerid_full = [i.text for i in sellerid_raw]
  sellerid_name = [i.split('by ')[1] for i in sellerid_full]
  sellerid = [i.split('\n')[0] for i in sellerid_name]
  shipping_full = raw_html.find_all('div', {'class' : 'shipping'})
  shipping = [i.text for i in shipping_full]
  origin_full = [i.split('\n ')[1] for i in shipping]
  origin = [i.split(': ', 1)[1] for i in origin_full]
  destination_full = [i.split('\n ')[2] for i in shipping]
  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' : 'price'})
  priceBTC = [i.text for i in price_raw]
  priceBTC = [i.replace('฿','') for i in priceBTC]
  priceBTC = [float(i) for i in priceBTC]
  rating_full = raw_html.find_all('span', {'class' : 'vendor_rating average'})  # Seller Rating. Out of 5.
  rating = [i.split('\n')[2] for i in sellerid_full]
  for i in range(0, len(rating)): 
    try:
      float(rating[i])
    except ValueError:
      rating[i] = 0
  for i in range(0, len(rating)): 
        rating[i] = float(rating[i])/20
  reviews_full = raw_html.find_all('div', {'class' : 'rating_count'}) # The number of customer reviews
  reviews = [i.text for i in reviews_full]
  for i in range(0, len(reviews)): 
    reviews[i] = reviews[i].replace('(','').replace(')','')
    reviews[i] = int(reviews[i])
  priceUSD = [] # Price in American Dollars
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  df['BTCUSD'] = df['Date'].map(exchange)
  df['PriceUSD'] = df['BTCUSD'] * df['PriceBTC'] # Price in American Dollars
  df = df.drop('BTCUSD', axis=1)
  return df

#cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-11-05/categories/art/items.html')
cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-10-04/categories/apparel/items_page=11.html')
#cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-10-04/categories/drugs-stimulants/items.html')

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,Rolex - Lady Datejust 36mm Pink [Replica],RepAAA,135.484,0.416245,4.65,0,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-10-04
1,Hublot Maradona Replica,AsianVixen,167.088,0.51334,4.45,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
2,Prada Calf Leather Twin Pocket Glace Tote Bag ...,FoxyGirl,334.176,1.02668,4.5,0,China,Worldwide,Apparel,None,SilkRoad2,2014-10-04
3,IWC - Portuguese Chronograph RG Silver Dial [A...,RepAAA,362.82,1.11468,4.65,0,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-10-04
4,█GOLD FLEX BODY ARMOR █ CASUAL █CONCEALED█,LegalGoods,620.613,1.90669,0,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
5,Chanel Beige Ballerina Ballet Flats Shoes Repl...,FoxyGirl,171.862,0.528007,4.5,0,China,Worldwide,Apparel,None,SilkRoad2,2014-10-04
6,Hublot F1 Quartz Replica,AsianVixen,157.54,0.484006,4.45,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
7,Rolex Watches Women 187 ID: 43406,gotmilkreplica,128.896,0.396005,4.15,0,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-10-04
8,Rolex DateJust Zebra Replica,AsianVixen,171.862,0.528007,4.45,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
9,Custom Listing for W...h,FoxyGirl,176.636,0.542674,4.5,0,China,Worldwide,Apparel,None,SilkRoad2,2014-10-04


In [74]:
# Create empty list to populate
data = []

# Specify file directory with folders for different scraping sessions
datedir = "D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2"

# Adds html ext to all files. Applies file cleaning function to all files from each scraping session and appends to one of two dataframes.
for scrapedate in tqdm(os.listdir(datedir)):
    directory = datedir +'/' + scrapedate + '/categories/'
    for foldername in os.listdir(directory):
        folder = directory + foldername # Concatenate directory path and filename for reading into cleanhtml function
#       converthtmlsr2(folder)
        for filename in os.listdir(folder):
            file = folder + '/' + filename
            if scrapedate <= '2014-08-17':
                filedf = cleanhtmlsr2(file)  # Extract dataframe of required variables from each file
                data.append(filedf)  # Append empty list with each dataframe generated. Results in list of dataframes.
            elif '2014-08-18' <= scrapedate:
                filedf2 = cleanhtmlsr2part2(file)  # Extract dataframe of required variables from each file
                data.append(filedf2)
df = pd.concat(data)  # Concatenate list of dataframes into single dataframe 

100%|██████████| 53/53 [31:27<00:00, 35.62s/it]


In [76]:
df[20000:526748]
#df.duplicated()

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
24,A Guide to Internet Security- Becoming an Uebe...,profesorhouse,35.6021,0.059951,None,None,Belgium,Undeclared,Digital,Goods,SilkRoad2,2014-07-26
0,A Guide to Internet Security- Becoming an Uebe...,profesorhouse,35.6021,0.059951,None,None,Belgium,Undeclared,Digital,Goods,SilkRoad2,2014-07-26
1,"Free VPN For Windows, Mac & Linux",cyberzen,6.10302,0.010277,None,None,Undeclared,Undeclared,Digital,Goods,SilkRoad2,2014-07-26
2,[Web Traffic] How to generate MAXIMUM Web Traffic,cyberzen,10.1721,0.017129,None,None,Undeclared,Undeclared,Digital,Goods,SilkRoad2,2014-07-26
3,"Cookies, Flash Cookies, Paypal, eBay Cleaner ...",cyberzen,5.08575,0.008564,None,None,Undeclared,Undeclared,Digital,Goods,SilkRoad2,2014-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...
15,2000 Xanax GG249 Bars - These are banging - $2...,gotmilk,2073.78,5.83885,4.55,0,India,Worldwide,Prescription,Relaxants,SilkRoad2,2014-10-24
16,XANAX 30x1mg █ original genuine pfizer blister...,sleephaeven,65.6662,0.184887,4.25,0,Germany,Worldwide,Prescription,Relaxants,SilkRoad2,2014-10-24
17,GG249 Xanax Bars - 2000 Bars - EMS,gotmilk,2073.78,5.83885,4.55,0,India,Worldwide,Prescription,Relaxants,SilkRoad2,2014-10-24
18,Rivotril 2mg Clonazepam - 500 pills - $550 us...,gotmilk,570.29,1.60568,4.55,0,India,Worldwide,Prescription,Relaxants,SilkRoad2,2014-10-24


In [78]:
df.to_excel(r'D:\Darkweb Data\dnmarchives\silkroad2.tar\mastersr2.xlsx', index = False)

In [82]:
df.isnull().sum()

Title            1786
Sellerid         1790
PriceUSD         1794
PriceBTC            8
Rating         326200
Reviews        326200
Origin           1791
Destination      1792
Category         1786
Subcategory      1786
Market           1786
Date             1786
dtype: int64